## High End Resale GL Report Processor

GL Reports are a nightmare. The product of month's worth of transactions piled up into a single, horrifically messy text document, which Cougar Mountain can barely keep straight. The formatting is incredibly inconsistent. The fact that it relies on unreliable indentation to clarify between debits and credits. Arbitrary character limits. Hardcoded indices.

Anyway, let's do our best to convert this horrific data display into a readable Excel document, by first going through JSON. Or maybe namedtuples.

Note: I actually did this already. But I lost the code, so now I have to remake it. A lesson for the curious: always commit your code whenever it works. It will save you from lots of pain and sadness.

### Layout

The GL report is split into thousands of pages, each of which is separated by seven newlines. Each page contains a header, which is relatively useless information, and then there are three newlines leading into one or more section parts. Here's where the hard part starts.

Each section contains a header line, which contains a (possibly decimal) number, four spaces, and a title. The title holds information about the type of section, and may contain a location (for example, Abdn or Hby). After the header, there is a completely useless line of column titles (same for every section, so it contains no useful information) and then a number of entries, which are usually one to three lines of information which may or may not be important, usually containing some amount of money (which can be positive or negative, but the only way to know for sure is unreliable indentation). If the section ends on the page, there will also be a "balance forward" row, then a row showing the total debits or credits of the entire section. If the section does not end, the next part of the section will have another header and then continue showing entries. All entries are in chronological order within sections, and at the end of every month, there is a "totals for" row showing the totals for the month. This can be used to check if entries were read correctly, and is also just good information to store.

### The Game Plan

Different section types require different types of processing. That said, there are important commonalities that we can and should take advantage of. For example, all sections have roughly the same high-level layout, they each are composed of some amount of entries (all of which are relatively similar) and of course Totals for processing is consistent across all section types. The first time I did this, I maintained a global Info dictionary that was updated by a swarm of line processing functions. Said line processing functions also updated the line index of a while loop that iterated line through line, and the while loop also checked for special lines, carrying progress across pages. I'll probably do something similar this time around because I don't think I can come up with a significantly better methodology. On the other hand, there are certainly imperfections that I can iron out. (Like the horrific inconsistency between different kinds of entries. That really isn't necessary here.)

In [3]:
# Imports
import pandas as pd
from datetime import datetime, timedelta
from gl_processor import GLProcessor

In [4]:
# Read file, split into pages
yr = (datetime.now() - timedelta(days=60)).year
with open(f"GL{yr}.txt", 'r') as f:
    report = f.read()
    pages = [p.strip() for p in report.split('\n' * 7)]

In [5]:
gl = GLProcessor(f"GL{yr}.txt")
gl.process()

Inventory: 639 transactions
Balance Forward: [Decimal('49505.86'), Decimal('192498.18')]

COGS - Broken item w/o: 281 transactions
Balance Forward: [Decimal('21828.26'), Decimal('15926.07')]

COGS - Missing item w/o: 2 transactions
Balance Forward: [Decimal('18.00'), Decimal('12.00')]

Cash chequing account-Out: 116 transactions
Balance Forward: [Decimal('1814358.06'), Decimal('326492.12')]

TD Business Investor Account: 9 transactions
Balance Forward: [Decimal('230865.81'), Decimal('1046279.80')]

US $ Chequing Account: 4 transactions
Balance Forward: [Decimal('795983.57'), Decimal('780190.40')]

US $ Account Exchange: 4 transactions
Balance Forward: [Decimal('230296.23'), Decimal('225021.55')]

Accounts Receivalbe: 73 transactions
Balance Forward: [Decimal('146939.16'), Decimal('167313.89')]

GST Input Credit: 55 transactions
Balance Forward: [Decimal('1571.85'), Decimal('2737.65')]

Prepaid: 5 transactions
Balance Forward: [Decimal('0.00'), Decimal('32942.66')]

Accounts Payable: 5 